In [1]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/dataset91/data.py", dst = "../working/data.py")
copyfile(src = "../input/dataset91/utils.py", dst = "../working/utils.py")
copyfile(src = "../input/dataset91/training.py", dst = "../working/training.py")
copyfile(src = "../input/dataset91/layers.py", dst = "../working/layers.py")
#copyfile(src = "../input/notes1/weights2.npy", dst = "../working/weights2.npy")
# import all our functions
#from data import load_data

'../working/layers.py'

In [2]:
!pip install pickle5
import pickle5 as pickle
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
from scipy.interpolate import splrep
from IPython.core.display import display_html
from keras.models import load_model
from utils import np_haversine, density_map, get_clusters, plot_embeddings
from data import load_data
from training import start_new_session, process_features, create_model

# Display plots inline
%matplotlib inline

# Fix random seed
np.random.seed(42)

     |████████████████████████████████| 256 kB 896 kB/s            


In [3]:
# Estimate the GPS clusters
print("Estimating clusters...")
#clusters = get_clusters(data.train_labels)
n_epochs=100
batch_size=200
save_prefix='mymodel'
config=3

Estimating clusters...


In [4]:
clusters_cache = '/kaggle/input/dataset91/clusters.npy'
clusters=np.load(clusters_cache, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [5]:
file='/kaggle/input/dataset91/train0.npy'
train0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train1.npy'
train1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train2.npy'
train2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train3.npy'
train3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train4.npy'
train4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train5.npy'
train5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train6.npy'
train6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
train=[train0,train1,train2,train3,train4,train5,train6]

In [6]:
file='/kaggle/input/dataset91/validation0.npy'
validation0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation1.npy'
validation1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation2.npy'
validation2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation3.npy'
validation3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation4.npy'
validation4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation5.npy'
validation5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation6.npy'
validation6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
validation=[validation0,validation1,validation2,validation3,validation4,validation5,validation6]

TrajLSTM-LocationAttention

Fitur

In [7]:
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine
from data import load_data
from utils import get_clusters
from keras.models import Model

from tensorflow.compat.v1.keras.layers import LSTM   # CuDNNLSTM
import sys
sys.path.append('..')  # add parent directory to Python path for layers.py access
from layers import Attention, SelfAttention

def create_model(metadata,clusters,config):
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    # Quarter hour of the day embedding
    embed_quarter_hour = Sequential()
    embed_quarter_hour.add(Embedding(metadata['n_quarter_hours'], embedding_dim, input_length=1))
    embed_quarter_hour.add(Reshape((embedding_dim,1)))

    # Day of the week embedding
    embed_day_of_week = Sequential()
    embed_day_of_week.add(Embedding(metadata['n_days_per_week'], embedding_dim, input_length=1))
    embed_day_of_week.add(Reshape((embedding_dim,1)))

    # Week of the year embedding
    embed_week_of_year = Sequential()
    embed_week_of_year.add(Embedding(metadata['n_weeks_per_year'], embedding_dim, input_length=1))
    embed_week_of_year.add(Reshape((embedding_dim,1)))

    # Client ID embedding
    embed_client_ids = Sequential()
    embed_client_ids.add(Embedding(metadata['n_client_ids'], embedding_dim, input_length=1))
    embed_client_ids.add(Reshape((embedding_dim,1)))

    # Taxi ID embedding
    embed_taxi_ids = Sequential()
    embed_taxi_ids.add(Embedding(metadata['n_taxi_ids'], embedding_dim, input_length=1))
    embed_taxi_ids.add(Reshape((embedding_dim,1)))

    # Taxi stand ID embedding
    embed_stand_ids = Sequential()
    embed_stand_ids.add(Embedding(metadata['n_stand_ids'], embedding_dim, input_length=1))
    embed_stand_ids.add(Reshape((embedding_dim,1)))
    
    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)
    coords = Sequential()
    coords.add(Dense(1, input_dim=20))

    # Merge all the inputs into a single input layer
    mergedOut = Add()([embed_quarter_hour.output,
                embed_day_of_week.output,
                embed_week_of_year.output,
                embed_client_ids.output,
                embed_taxi_ids.output,
                embed_stand_ids.output,
                coords.output])
    
    #encoder_output, hidden_state, cell_state = LSTM(50, activation='tanh',input_shape=(None,None, 1),
    #                                                return_sequences=True,return_state=True)(mergedOut)
    #attention_input = [encoder_output, hidden_state]
    lstm_layer = tf.keras.layers.LSTM(200, return_sequences=True,return_state=True)
    if config != 0:
        encoder_output, hidden_state, cell_state,  *_ = Bidirectional(lstm_layer,merge_mode="sum")(mergedOut)
        attention_input = [encoder_output, hidden_state]
    else:
        encoder_output = LSTM(units=64)(mergedOut)

    # Optional Attention Mechanisms
    if config == 1:
        encoder_output, attention_weights = SelfAttention(size=64,
                                                      num_hops=10,
                                                      use_penalization=False)(encoder_output)
    elif config == 2:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='global')(attention_input)
        encoder_output = Flatten()(encoder_output)
    elif config == 3:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='local-m',
                                                  window_width=10,
                                                  score_function='general')(attention_input)
        encoder_output = Flatten()(encoder_output)
    
    #encoder_output=Dense(1)(encoder_output)
    # Determine cluster probabilities using softmax
    mergedOut=Dense(len(clusters))(encoder_output)
    mergedOut=Activation('softmax')(mergedOut)

    # Final activation layer: calculate the destination as the weighted mean of cluster coordinates
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    mergedOut=Activation(destination)(mergedOut)

    newModel = Model([embed_quarter_hour.input,
                embed_day_of_week.input,
                embed_week_of_year.input,
                embed_client_ids.input,
                embed_taxi_ids.input,
                embed_stand_ids.input,
                coords.input], mergedOut)
    #use lists if you want more than one input or output  
    
    # Compile the model
    optimizer = SGD(lr=0.1, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients

    newModel.compile(loss=tf_haversine, optimizer=optimizer)
    
    return newModel

GPS

Create Model dan Training

In [8]:
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)
tl_cache = '/kaggle/input/dataset91/train-labels.npy'
vl_cache = '/kaggle/input/dataset91/validation-labels.npy'
train_labels=np.load(tl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
validation_labels=np.load(vl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [9]:
config=3
model = create_model(metadata,clusters,config)
#c=0

2022-01-17 00:36:30.499707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:30.603943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:30.604666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:30.605895: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Parameter

Custom Callback

In [10]:
e=[]
f=[]
g=[]
for i in range(60):
    f.append(0)
    e.append(0)
    g.append(0)

In [11]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(e)
        if z>len(g):
            a=model.history.history['val_loss'][z-len(g)-1]
            b=model.history.history['loss'][z-len(g)-1]
            c=float(a)-float(b)
            if c<0:
                print("Sebelumnya Underfitting")
            else:
                print("Sebelumnya Overfitting")
        e.append(0)
            

In [12]:
class CustCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(f)
        file_path="./weights"+str(z)
        weight=model.get_weights()
        np.save(file_path, weight)
        f.append(0)

Load Saved Weight

In [13]:
file='../input/latb91fiturlocm/weights'+str(len(g)-1)+'.npy'
#file='./weights'+str(len(g)-1)+'.npy'
y=np.load(file,allow_pickle=True)
model.set_weights(y)

Training

In [14]:
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping

#g.append(0)
#e.append(0)
arc='LSTMnew91-b200-'+str(len(g))+'.log'
csv_logger = CSVLogger(arc, separator=',', append=False)
print("Creating model...")
start_new_session()
print("Train model...")
history=model.fit(train, train_labels,
        initial_epoch=len(g),epochs=100, batch_size=batch_size,
        validation_data=(validation, validation_labels)
        ,callbacks=[csv_logger,CustCallback(),MyCallback()])

T=True
while(T):
    T=True

Creating model...


2022-01-17 00:36:36.328249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:36.331284: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:36.331947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:36.332697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 00:36:36.333392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Train model...


2022-01-17 00:36:37.077386: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 61/100


...

2022-01-17 00:38:12.975913: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


7400/7400 [==============================] - 100s 13ms/step - loss: 1.9219 - val_loss: 2.0065
Epoch 62/100
  11/7400 [..............................] - ETA: 1:28 - loss: 1.9553

/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


7400/7400 [==============================] - 93s 13ms/step - loss: 1.9210 - val_loss: 2.0028
Sebelumnya Overfitting
Epoch 63/100
7400/7400 [==============================] - 94s 13ms/step - loss: 1.9205 - val_loss: 2.0049
Sebelumnya Overfitting
Epoch 64/100
7400/7400 [==============================] - 94s 13ms/step - loss: 1.9200 - val_loss: 2.0079
Sebelumnya Overfitting
Epoch 65/100
7400/7400 [==============================] - 93s 13ms/step - loss: 1.9186 - val_loss: 2.0079
Sebelumnya Overfitting
Epoch 66/100
7400/7400 [==============================] - 93s 13ms/step - loss: 1.9167 - val_loss: 2.0082
Sebelumnya Overfitting
Epoch 67/100
7400/7400 [==============================] - 94s 13ms/step - loss: 1.9170 - val_loss: 2.0275
Sebelumnya Overfitting
Epoch 68/100
7400/7400 [==============================] - 92s 12ms/step - loss: 1.9143 - val_loss: 2.0130
Sebelumnya Overfitting
Epoch 69/100
7400/7400 [==============================] - 93s 13ms/step - loss: 1.9138 - val_loss: 2.0019
Sebe

KeyboardInterrupt: 

In [ ]:
#metadata_cache = '/kaggle/input/dataset91/train.pickle'
#with open(metadata_cache, 'rb') as handle:
#    train = pickle.load(handle)
#metadata_cache = '/kaggle/input/dataset91/validation.pickle'
#with open(metadata_cache, 'rb') as handle:
#    validation = pickle.load(handle)
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/competition-test.pickle'
with open(metadata_cache, 'rb') as handle:
    competition_test = pickle.load(handle)
train_labels=np.load('/kaggle/input/dataset91/train-labels.npy')
validation_labels=np.load('/kaggle/input/dataset91/validation-labels.npy')
competition_test_labels=np.load('/kaggle/input/dataset91/competition-test-labels.npy')

In [ ]:
print("HASIL EVALUASI MODEL-INPUT 82 Full GPS")
print(" ")
test_predictions = model.predict(process_features(competition_test))
print("Mean Haversine Test:{}".format(np_haversine(test_predictions, competition_test_labels).mean()))
validation_predictions = model.predict(validation)
print("Mean Haversine Validation:{}".format(np_haversine(validation_predictions, validation_labels).mean()))
train_predictions = model.predict(train)
print("Mean Haversine Train:{}".format(np_haversine(train_predictions, train_labels).mean()))
print(" ")
y_train=train_labels
y_train_pred=train_predictions
y_valid=validation_labels
y_valid_pred=validation_predictions
y_test=competition_test_labels
y_test_pred=test_predictions
from sklearn import metrics
print("MSE train:{}".format(metrics.mean_squared_error(y_train, y_train_pred)))
print("MSE validation:{}".format(metrics.mean_squared_error(y_valid, y_valid_pred)))
print("MSE test:{}".format(metrics.mean_squared_error(y_test, y_test_pred)))
print(" ")
print("MAE score train:{}".format(metrics.mean_absolute_error(y_train, y_train_pred)))
print("MAE score validation:{}".format(metrics.mean_absolute_error(y_valid, y_valid_pred)))
print("MAE score test:{}".format(metrics.mean_absolute_error(y_test, y_test_pred)))
print(" ")
print("R2 score train:{}".format(metrics.r2_score(y_train, y_train_pred)))
print("R2 score validation:{}".format(metrics.r2_score(y_valid, y_valid_pred)))
print("R2 score test:{}".format(metrics.r2_score(y_test, y_test_pred)))

EVALUASI

FITUR